# 과제 1: 사용자의 취향을 기억하는 영화 추천 챗봇

## 기능
- 사용자가 좋아하는 장르를 기억
- 이미 시청한 영화를 기억
- 대화 기록을 기반으로 개인화된 추천 제공

## 구현 방식
- `messages` 리스트로 전체 대화 기록 유지
- 사용자 메시지(`role: "user"`)와 AI 응답(`role: "assistant"`)을 모두 append
- system prompt에서 이미 본 영화를 추천하지 않도록 지시

In [1]:
import openai
import dotenv

dotenv.load_dotenv()

client = openai.OpenAI()

In [2]:
SYSTEM_PROMPT = """당신은 친절한 영화 추천 전문가입니다.

당신의 역할:
1. 사용자의 선호 장르를 대화에서 파악하고 기억합니다.
2. 사용자가 이미 본 영화를 기억하고, 절대 다시 추천하지 않습니다.
3. 대화 맥락을 기반으로 개인화된 영화를 추천합니다.
4. 사용자가 자신의 취향이나 이미 본 영화를 물어보면 대화 기록에서 정확히 답합니다.

답변은 항상 한국어로, 자연스럽고 친근하게 해주세요.
추천할 때는 영화 제목과 함께 간단한 이유를 설명해 주세요."""


# 대화 기록을 저장하는 messages 리스트
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
]


def chat(user_message: str):
    """사용자 메시지를 보내고 AI 응답을 받는 함수"""
    print(f"👤 User: {user_message}\n")

    # 사용자 메시지를 대화 기록에 추가
    messages.append({"role": "user", "content": user_message})

    # AI 응답 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    assistant_message = response.choices[0].message.content

    # AI 응답을 대화 기록에 추가
    messages.append({"role": "assistant", "content": assistant_message})

    print(f"🤖 AI: {assistant_message}\n")
    print(f"--- 현재 대화 기록: {len(messages)}개 메시지 (system 포함) ---")

## 챗봇 테스트

In [3]:
# 대화 1: 선호 장르 알려주기
chat("나는 SF 영화를 좋아해")

👤 User: 나는 SF 영화를 좋아해

🤖 AI: SF 영화를 좋아하시군요! 정말 흥미로운 장르죠. 이미 본 SF 영화들이 있다면 말씀해 주시면 그걸 바탕으로 더 나은 추천을 드릴 수 있을 것 같아요. 어떤 영화들을 보셨나요?

--- 현재 대화 기록: 3개 메시지 (system 포함) ---


In [4]:
# 대화 2: 이미 본 영화 알려주기
chat("인셉션이랑 인터스텔라는 이미 봤어")

👤 User: 인셉션이랑 인터스텔라는 이미 봤어

🤖 AI: 인셉션과 인터스텔라를 보셨군요! 두 영화 모두 다층적인 스토리와 깊은 주제를 가지고 있죠. 그렇다면 **"더 마션(2015)"**을 추천해드리고 싶어요.  

이 영화는 한 우주 비행사가 화성에 고립되어 생존하는 과정을 그린 이야기로, 과학적 요소와 긴장감이 잘 어우러져 있습니다. 또한 따뜻한 유머도 있어서 긴장 속에서도 웃음을 잃지 않게 해줍니다. 꼭 한 번 보세요!

--- 현재 대화 기록: 5개 메시지 (system 포함) ---


In [5]:
# 대화 3: 영화 추천 요청 (이미 본 영화 제외해야 함)
chat("오늘 밤에 뭐 볼지 추천해 줄래?")

👤 User: 오늘 밤에 뭐 볼지 추천해 줄래?

🤖 AI: 물론이죠! 오늘 밤 볼 만한 SF 영화로 **"엑스 마키나(2014)"**를 추천할게요. 이 영화는 인공지능과 인간의 관계를 탐구하는 과정을 담고 있어서, 흥미로운 철학적 질문들도 던지는 작품이에요. 감정적으로도 아주 긴장감 넘치는 전개가 매력적이랍니다. 재미있게 감상하세요!

--- 현재 대화 기록: 7개 메시지 (system 포함) ---


In [6]:
# 대화 4: 메모리 확인 - 선호 장르와 이미 본 영화를 기억하는지 테스트
chat("내가 좋아하는 장르랑 이미 본 영화가 뭐라고 했지?")

👤 User: 내가 좋아하는 장르랑 이미 본 영화가 뭐라고 했지?

🤖 AI: 당신은 SF 영화를 좋아하시고, "인셉션"과 "인터스텔라"를 이미 보셨다고 하셨어요. 혹시 추가로 궁금한 점이나 다른 추천을 원하신다면 말씀해 주세요!

--- 현재 대화 기록: 9개 메시지 (system 포함) ---
